# Rozpoznawanie mowy modelami end-to-end

Do tego ćwiczenia użyjemy bilioteki [Transformers](https://huggingface.co/) opartej na toolkicie PyTorch. Ten konkretny zeszyt korzysta z możliwości liczenia na GPU, choć można też wszystko robić na CPU ale będzie działało nieco wolniej.

Polecam włączyć GPU w "Runtime/Change runtime type" i sprawdzić jaką kartę otrzymaliśmy (oraz ile ma pamięci) poleceniem `!nvidia-smi`:

Ściągnijmy i rozpakujmy kilka przykładowych nagrań z adresu http://users.pja.edu.pl/~danijel/zum/nagrania.tar.gz :

In [103]:
!wget http://users.pja.edu.pl/~danijel/zum/nagrania.tar.gz
!tar xvf nagrania.tar.gz

--2022-10-27 18:46:58--  http://users.pja.edu.pl/~danijel/zum/nagrania.tar.gz
Resolving users.pja.edu.pl (users.pja.edu.pl)... 91.230.222.21, 2001:67c:23f4::ab00
Connecting to users.pja.edu.pl (users.pja.edu.pl)|91.230.222.21|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2684755 (2.6M) [application/x-gzip]
Saving to: ‘nagrania.tar.gz.1’

nagrania.tar.gz.1   100%[===================>]   2.56M  1.06MB/s    in 2.4s    

2022-10-27 18:47:01 (1.06 MB/s) - ‘nagrania.tar.gz.1’ saved [2684755/2684755]

nagrania/
nagrania/Danio_part1_2.wav
nagrania/Mike_part1_2.wav
nagrania/text
nagrania/Marek_part1_6.wav
nagrania/Adam_part1_0.wav
nagrania/Marek_part1_9.wav
nagrania/Szczepan_part1_9.wav
nagrania/Jacek_part1_4.wav
nagrania/Adam_part1_1.wav
nagrania/Filip_part1_1.wav
nagrania/Marek_part1_2.wav


Zainstalujmy też następujące biblioteki poleceniem `!pip install <nazwa>`:

* `https://github.com/huggingface/transformers/archive/refs/heads/master.zip` - biblioteka Transformers instalowana z Githuba
* `https://github.com/kensho-technologies/pyctcdecode/archive/refs/heads/main.zip` - biblioteka do dekodowania modelem języka
* `https://github.com/kpu/kenlm/archive/master.zip` - biblioteka do obsługi modelu języka w dekoderze
* `wavio` - biblioteka do wczytywania plików audio
* `jiwer` - biblioteka do liczenia WER
* `arpa` - mała biblioteka do wczytywania i oglądania modeli języka

In [2]:
!pip install https://github.com/huggingface/transformers/archive/refs/heads/master.zip
!pip install https://github.com/kensho-technologies/pyctcdecode/archive/refs/heads/main.zip
!pip install https://github.com/kpu/kenlm/archive/master.zip
!pip install wavio
!pip install jiwer
!pip install arpa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     \ 13.6 MB 3.2 MB/s
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.6 MB 3.0 MB/s 
     |████████████████████████████████| 163 kB 72.1 MB/s 
  Created wheel for transformers: filename=transformers-4.24.0.dev0-py3-none-any.whl size=5427024 sha256=21e878b755a25cf91c060105c8783a158e32e08cb24747c3d64d23d919ecf2fe
  Stored in directory: /tmp/pip-ephem-wheel-cache-m16rgbiu/wheels/92/7e/03/a16a3d5eb8a53365edd738b739018eb170666f47b01a1776ad
Successfully built transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     - 217 kB 1.2 MB/s
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 395 kB 6.9 MB/s 
  C

Dodatkowo, ściągnijmy i rozpakumy program SRILM z adresu http://users.pja.edu.pl/~danijel/zum/srilm-bin.tar.xz

Potem ustaw symlinki do programów `ngram` oraz `ngram-count` w `/usr/local/bin` - żeby łatwiej korzytać z tych programów:

In [3]:
!wget http://users.pja.edu.pl/~danijel/zum/srilm-bin.tar.xz
!tar xvf srilm-bin.tar.xz
!ln -sf ${PWD}/bin/i686-m64/ngram /usr/local/bin/ngram
!ln -sf ${PWD}/bin/i686-m64/ngram-count /usr/local/bin/ngram-count

--2022-10-27 18:19:52--  http://users.pja.edu.pl/~danijel/zum/srilm-bin.tar.xz
Resolving users.pja.edu.pl (users.pja.edu.pl)... 91.230.222.21, 2001:67c:23f4::ab00
Connecting to users.pja.edu.pl (users.pja.edu.pl)|91.230.222.21|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37056672 (35M) [application/x-xz]
Saving to: ‘srilm-bin.tar.xz’

srilm-bin.tar.xz    100%[===================>]  35.34M  9.01MB/s    in 4.7s    

2022-10-27 18:19:57 (7.57 MB/s) - ‘srilm-bin.tar.xz’ saved [37056672/37056672]

bin/
bin/merge-batch-counts
bin/make-batch-counts
bin/compute-sclite-nbest
bin/pfsg-from-ngram
bin/rescore-acoustic
bin/nbest-error
bin/search-rover-combo
bin/rexport.gnumake
bin/make-multiword-pfsg
bin/rescore-minimize-wer
bin/rescore-reweight
bin/i686-m64/
bin/i686-m64/sentid-to-sclite
bin/i686-m64/reverse-lm
bin/i686-m64/select-vocab
bin/i686-m64/subset-context-ngrams
bin/i686-m64/maxalloc
bin/i686-m64/make-kn-counts
bin/i686-m64/wlat-to-pfsg
bin/i686-m64/wlat-stats


Zaimportuj wszystkie potrzebne niżej komponenty z zainstlowanych bibliotek:

In [4]:
%pylab inline

from pathlib import Path
from tqdm import tqdm

import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from pyctcdecode import build_ctcdecoder
from wavio import read
import jiwer
import arpa

Populating the interactive namespace from numpy and matplotlib


## Instalacja modeli

Użyj metodę `from_pretrained` na klasach `Wav2Vec2Processor` oraz `Wav2Vec2ForCTC` żeby ściągnąć i zainstalować modele o nazwie: `facebook/wav2vec2-base-10k-voxpopuli-ft-pl`

Dodatkowo, zastosuj metodę `.to('cuda')` na głównym modelu (CTC) żeby go "przerzucić" na GPU:

In [5]:
processor=Wav2Vec2Processor.from_pretrained('facebook/wav2vec2-base-10k-voxpopuli-ft-pl')
model=Wav2Vec2ForCTC.from_pretrained('facebook/wav2vec2-base-10k-voxpopuli-ft-pl').to('cuda')

Downloading:   0%|          | 0.00/213 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/138 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/381 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/378M [00:00<?, ?B/s]

## Wczytanie plików audio

Wczytaj wszystkie pliki audio do słownika `files` mapującego identyfikatory plików na ich jednowymiarowe wersje zpisane typem `float32`:

In [89]:
files={}
for f in Path('nagrania').glob('*.wav'):
  data=read(str(f))
  files[f.stem]=data.data.squeeze().astype('float32')

In [90]:
files

{'Adam_part1_1': array([ 43.,  41.,  31., ..., -94., -99., -97.], dtype=float32),
 'Danio_part1_2': array([27., 32., 25., ..., -6., -5., -5.], dtype=float32),
 'Adam_part1_0': array([101.,  99.,  98., ..., -15., -18., -16.], dtype=float32),
 'Marek_part1_6': array([-635., -677., -687., ..., -766., -831., -770.], dtype=float32),
 'Marek_part1_2': array([ 217.,  196.,  190., ...,  -54., -114.,  -78.], dtype=float32),
 'Jacek_part1_4': array([-309., -314., -327., ..., -239., -240., -233.], dtype=float32),
 'Mike_part1_2': array([-146., -152., -142., ...,    6.,   14.,   -5.], dtype=float32),
 'Filip_part1_1': array([-1205., -1226., -1326., ..., -1007.,  -990., -1044.], dtype=float32),
 'pi': array([  84.,   84.,   81., ..., 2969., -445., -621.], dtype=float32),
 'Paulina1': array([-3461., -3461., -3466., ..., -3480., -3420., -3411.], dtype=float32),
 'ping': array([-955., -955., -957., ..., -933., -937., -935.], dtype=float32),
 'Paul': array([362., 362., 358., ..., 589., 226., 618.], dty

In [91]:
Fs=data.rate
for name,d in files.items():
  print(f'{name}: {d.size/Fs:0.2f}s')

Adam_part1_1: 8.27s
Danio_part1_2: 8.35s
Adam_part1_0: 8.83s
Marek_part1_6: 9.12s
Marek_part1_2: 10.34s
Jacek_part1_4: 8.24s
Mike_part1_2: 8.65s
Filip_part1_1: 10.52s
pi: 9.00s
Paulina1: 4.00s
ping: 9.00s
Paul: 9.00s
Marek_part1_9: 11.89s
Paulina3: 5.00s
Paulina2: 4.00s
Szczepan_part1_9: 9.79s


## Przetwarzanie podstawowym modelem ASR

Dla wszystkich plików zrób co następuje:
1. przepuść audio przez procesor podając dodatkowo argumenty `sampling_rate`, `return_tensors='pt'` oraz `padding=True`
2. przrzuć wynik na GPU poleceniem `to('cuda')` gdyż model jest na karcie a dane nie są
3. przepuść składową `.input_values` przez model i zapisz wynik do zmiennej
4. składowa wyniku `.logits` zawiera wzystkie wagi wyjściowe modelu - zastosuj na niej `torch.argmax` po ostatnim wymiarze
5. w wyniku otrzymasz listę identyfikatorów którą można zamienić na słow funkcją `batch_decode` procesora
6. zapisz wynik do słownika `trans`

In [92]:
trans={}
for name,data in tqdm(files.items()):
  feats=processor(data,sampling_rate=Fs,return_tensors='pt',padding=True).to('cuda')
  out=model(input_values=feats.input_values)
  predicted_ids=torch.argmax(out.logits,dim=-1)
  sent=processor.batch_decode(predicted_ids)[0]
  trans[name]=sent

100%|██████████| 16/16 [00:00<00:00, 19.26it/s]


In [93]:
trans

{'Adam_part1_1': 'dwaa siedem pięć trzy zero jeden sześć dziewięć jeden pięćsiedem sześć pięć trzyzro sześć jeden czteryzr',
 'Danio_part1_2': 'siedem dwazer o dwa dziewięć dwa jedenz siedem jeden pięć dziewięć dwa pięć siedem dziewięć pięć pięć trzy cztery cztery',
 'Adam_part1_0': 'siedem siedem sześć sześć sześć osiem siedem trzy osiem ty osiem sześć siedem pięć sześć pięć pięć trzy pięć zero',
 'Marek_part1_6': 'osiem dziewięć sześć sześć jeden dwa jeden dwa trzy dwa w sześć zero cztery sześć sześć da dziewięć trzy trzy osiem',
 'Marek_part1_2': 'siedem osiem trzy osiem pięć zerotytzy pięć osiem siedem cztery jeden zero dziewięć trzy osiem jeden',
 'Jacek_part1_4': 'pzerojeden dwa dwa osiem siedem dziewięć dziewięć sześć cztery pięć cztery jeden jeden trzy osiem dziewięćseto sześć jeden',
 'Mike_part1_2': 'jeden dziewięć osiem cztery pięć dziewięćzy dwa dziewięćzeto sześć siedem osiem osiemzetty osiem cztery dziewięć sześć',
 'Filip_part1_1': 'trzy dwa trzy zero osiem sześć jeden d

## Weryfikacja jakości

Wczytaj plik `nagrania/text` do słownika `ref`:

In [94]:
ref={}
with open('nagrania/text') as f:
  for l in f:
    tok=l.strip().split()
    ref[tok[0]]=' '.join(tok[1:])

In [95]:
h=[]
r=[]

for name in trans.keys():
  print(f'>>{name}')
  print(trans[name])
  print(ref[name])
  print('')

  h.append(trans[name])
  r.append(ref[name])

print(jiwer.compute_measures(r,h))

>>Adam_part1_1
dwaa siedem pięć trzy zero jeden sześć dziewięć jeden pięćsiedem sześć pięć trzyzro sześć jeden czteryzr
dwa siedem pięć trzy zero jeden sześć dziewięć jeden pięć siedem sześć pięć trzy zero sześć jeden cztery zero zero

>>Danio_part1_2
siedem dwazer o dwa dziewięć dwa jedenz siedem jeden pięć dziewięć dwa pięć siedem dziewięć pięć pięć trzy cztery cztery
siedem dwa zero dwa dziewięć dwa jeden siedem jeden pięć dziewięć dwa pięć siedem dziewięć pięć pięć trzy cztery cztery

>>Adam_part1_0
siedem siedem sześć sześć sześć osiem siedem trzy osiem ty osiem sześć siedem pięć sześć pięć pięć trzy pięć zero
siedem siedem sześć sześć sześć osiem siedem trzy osiem trzy osiem sześć siedem pięć sześć pięć pięć trzy pięć zero

>>Marek_part1_6
osiem dziewięć sześć sześć jeden dwa jeden dwa trzy dwa w sześć zero cztery sześć sześć da dziewięć trzy trzy osiem
osiem dziewięć sześć sześć jeden dwa jeden dwa trzy dwa sześć zero cztery sześć sześć dwa dziewięć trzy trzy osiem

>>Marek_part

## Modelowanie języka

Modele języka trenujemy na tzw. korpusach tekstów. Taki korpus to po prostu lista poprawnych zdań w wybranym języku - jedno zdanie na linię. Zróbmy taki miniaturowy przykładowy korpus zawierający 3 zdania i zapiszmy do pliku `test.txt`:

Do utworzenia modelu języka użyjemy programu `ngram-count` z pakietu SRILM. Program ten ma mnóstwo opcji, które można obejrzeć uruchamiając go z opcją `-help`. Nas będą interesować następujące ustawienia:

 * `-text test.txt` - tym ustawimy plik źródłowy z korpusem
 * `-order 3` - tym ustawimy to, że chcemy mieć model 3-gramowy
 * `-wbdiscount` - użyjemy metodę wygładzania Witten-Bell
 * `-lm out.arpa` - wynik zapiszemy do pliku `out.arpa`

Jedną z podstatowych miar jakości modelu języka to tzw. *perplexity*. Liczymy go stotując wytrenowany model języka na niezależnym zbiorze testowym. Zróbmy przykładowy zbiór zawierający jedno zdanie i zapiszmy w pliku `eval.txt`:

## Model języka do rozpoznawania ciągów liczb

Stwórz listę cyfr: `['zero','jeden','dwa','trzy','cztery','pięć','sześć','siedem','osiem','dziewięć', 'krzyżyk', 'kółko', 'wstaw', 'pole']`

Wygeneruj do pliku `digits.txt` 100 linii po 10 losowych cyfr każda:

In [96]:
digits=['zero','jeden','dwa','trzy','cztery','pięć','sześć','siedem','osiem','dziewięć', 'krzyżyk na', 'kółko na dwa', 'wstaw', 'pole', 'kółko', 'krzyżyk', 'zaznacz' 'na']
with open('digits.txt','w') as f:
  for l in range(10000):
    f.write(' '.join([digits[x] for x in randint(0,17,10)])+'\n')

In [97]:
!head -n 22 digits.txt

zaznaczna zero krzyżyk cztery krzyżyk kółko sześć osiem zaznaczna krzyżyk
kółko na dwa krzyżyk siedem krzyżyk krzyżyk na dwa wstaw sześć krzyżyk na kółko
pięć wstaw krzyżyk na krzyżyk na sześć kółko na dwa trzy trzy sześć osiem
dwa osiem krzyżyk wstaw osiem trzy krzyżyk krzyżyk sześć osiem
zero pole kółko na dwa jeden kółko zaznaczna dwa dziewięć zero jeden
dwa krzyżyk na kółko na dwa kółko kółko krzyżyk cztery wstaw zero kółko na dwa
dwa dwa trzy krzyżyk na zaznaczna pole dwa cztery kółko krzyżyk na
jeden cztery pole zaznaczna krzyżyk zaznaczna krzyżyk kółko osiem pięć
krzyżyk na krzyżyk krzyżyk na jeden pięć krzyżyk na cztery pole krzyżyk siedem
kółko na dwa zero sześć dwa dwa dwa kółko dwa krzyżyk krzyżyk
pole zero pięć cztery sześć osiem zero trzy pole jeden
krzyżyk sześć kółko jeden siedem osiem wstaw kółko na dwa osiem pięć
zaznaczna kółko trzy wstaw trzy dziewięć zero wstaw wstaw krzyżyk
zaznaczna kółko trzy osiem kółko jeden sześć pole zaznaczna dziewięć
pole sześć krzyżyk na p

Zrób z tego pliku model języka `digits.arpa`:

In [98]:
!ngram-count -text digits.txt -order 3 -wbdiscount -unk -lm transcription.arpa

## Dekodowanie wygenerowanym modelem języka

Użyj `processor.tokenizer.get_vocab()` żeby dostać posortowaną (wg identyfikatorów) listę tokenów w modelu. Zamień separator słów (token `'|'`) na spację:

In [99]:
tokens=[x[0] for x in sorted(processor.tokenizer.get_vocab().items(),key=lambda x:x[1])]
print(tokens)
tokens[4]=' '
print(tokens)

['<pad>', '<s>', '</s>', '<unk>', '|', 'e', 'a', 'i', 'o', 'n', 'z', 'r', 'w', 's', 't', 'c', 'y', 'p', 'd', 'k', 'm', 'j', 'u', 'l', 'b', 'g', 'ł', 'h', 'ą', 'ę', 'ż', 'ó', 'ś', 'ć', 'f', 'ń', 'ź', 'v', 'x', 'q', '1']
['<pad>', '<s>', '</s>', '<unk>', ' ', 'e', 'a', 'i', 'o', 'n', 'z', 'r', 'w', 's', 't', 'c', 'y', 'p', 'd', 'k', 'm', 'j', 'u', 'l', 'b', 'g', 'ł', 'h', 'ą', 'ę', 'ż', 'ó', 'ś', 'ć', 'f', 'ń', 'ź', 'v', 'x', 'q', '1']


Użyj metody `build_ctcdecoder` żeby storzyć dekoder - podaj listę tokenów, plik ARPA oraz parametry `alpha=2.0` oraz `beta=-1.0`:

In [100]:
decoder=build_ctcdecoder(tokens,'transcription.arpa',alpha=2.0,beta=-1.0)

Powtórz proces dekodowania, ale zamiast liczyć `torch.argmax` przekaż wynik modelu do decodera. Metoda `decoder.decode` potrzebuje listy w postaci NumPy, więc trzeba ją zamienić używając poleceń: `out.logits.cpu().detach().numpy()`:

In [101]:
trans={}
for name,data in tqdm(files.items()):
  feats=processor(data,sampling_rate=Fs,return_tensors='pt',padding=True).to('cuda')
  out=model(input_values=feats.input_values)
  sent=decoder.decode(out.logits.cpu().detach().numpy()[0])
  trans[name]=sent

100%|██████████| 16/16 [00:02<00:00,  6.21it/s]


Na samym końcu, wyświetl wyniki i ponownie wylicz WER:

In [102]:

h=[]
r=[]

for name in trans.keys():
  print(f'>>{name}')
  print(trans[name])
  print(ref[name])
  print('')

  h.append(trans[name])
  r.append(ref[name])

print(jiwer.compute_measures(r,h))

>>Adam_part1_1
dwa siedem pięć trzy zero jeden sześć dziewięć jeden pięć siedem sześć pięć trzy sześć jeden cztery
dwa siedem pięć trzy zero jeden sześć dziewięć jeden pięć siedem sześć pięć trzy zero sześć jeden cztery zero zero

>>Danio_part1_2
siedem dwa zero dwa dziewięć dwa jeden siedem jeden pięć dziewięć dwa pięć siedem dziewięć pięć pięć trzy cztery cztery
siedem dwa zero dwa dziewięć dwa jeden siedem jeden pięć dziewięć dwa pięć siedem dziewięć pięć pięć trzy cztery cztery

>>Adam_part1_0
siedem siedem sześć sześć sześć osiem siedem osiem osiem sześć siedem pięć sześć pięć pięć trzy pięć
siedem siedem sześć sześć sześć osiem siedem trzy osiem trzy osiem sześć siedem pięć sześć pięć pięć trzy pięć zero

>>Marek_part1_6
osiem dziewięć sześć sześć jeden dwa jeden trzy sześć zero cztery sześć sześć dziewięć trzy trzy osiem
osiem dziewięć sześć sześć jeden dwa jeden dwa trzy dwa sześć zero cztery sześć sześć dwa dziewięć trzy trzy osiem

>>Marek_part1_2
siedem osiem trzy osiem pięć

# Praca domowa

## - na ocenę 3

Zastosuj powyższe rozwiązanie na swoich nagraniach i wyświetl wyniki.

## - na ocenę 4

Dokonaj optymalizacji procesu w celu polepszenia wyniku (pokaż ile się udało poprawić). Spróbuj następujące rzeczy:

- różne modele języka
- różne parametry dekodera
- inne modele z serwisu Huggingface

## - na ocenę 5

Połącz wynik rozpoznawania mowy z jakimś innym rozwiązaniem albo aplikacją.

Na przykład: 
* zrób prosty kalkulator głosowy
* zrób demonstracyjny dialog do zamawiania pizzy
* połącz rozwiązanie z jakimś innym modelem NLP

Rzuć okiem na https://huggingface.co/tasks oraz https://github.com/jonatasgrosman/huggingsound
